In [2]:
from moviepy.editor import AudioFileClip, VideoFileClip
from pydub import AudioSegment, effects
import stable_whisper
import pandas as pd
from yt_dlp import YoutubeDL
import csv
from natasha import (
    Segmenter,
    Doc
)

segmenter = Segmenter()

#Разделение на предложение
def split_by_sentences(text):
        doc = Doc(text)
        doc.segment(segmenter)

        return [x.text for x in doc.sents]

#Обрезка по таймкодам
def clip_filter(start, end, subtitles):
    return list(filter(lambda x: start<=x['start'] and x['start'] <= end, subtitles))  


In [ ]:
path = "/kaggle/input/vseros/"

# data = pd.read_csv(path+"data.csv", delimiter=',')
data = pd.read_csv("data.csv", delimiter=',')
data

In [ ]:
model = stable_whisper.load_model('tiny')

dataset = pd.DataFrame({'Sentense': {}, 'Tag' : {}})
for video in data['video']:
    print(video)
    #Скачиваем видео
    video_id = video.split("?v=")[1]
    ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    'outtmpl': f"audio{video_id}"
}
    YoutubeDL(ydl_opts).download(video)
    #Speech2Text
    result = model.transcribe(f'audio{video_id}.m4a')
    result.to_tsv('audio.tsv')

    #Форматируем
    subtitles = []
    read_tsv = csv.reader(open("audio.tsv"), delimiter="\t")
    for row in read_tsv:
        if len(row) > 0:
            subtitles.append({
                "start": int(row[0]) / 1000,
                "end": int(row[1]) / 1000,
                "text": row[-1]
            })

    tags = data[data['video'] == video]['tags'].values[0].replace('-', ' ').split(' ')

    sec_tags = []
    for tag in tags:
        min_sec = tag.split(':')
        if len(min_sec) == 3:
            sec_tags.append(int(min_sec[0])*60*60 + int(min_sec[1])*60 + int(min_sec[2]))
        else:
            sec_tags.append(int(min_sec[0])*60 + int(min_sec[1]))

    #Размечаем предложения
    for i in range(len(sec_tags)-1):
        is_interesting = (i % 2 == 0)
        start, end = sec_tags[i], sec_tags[i+1]
        
        clip_tags = clip_filter(start, end, subtitles)
        clip_text = ''.join([x['text']+' ' for x in clip_tags])
        clip_sentences = split_by_sentences(clip_text)

        rows_list = []
        for sentence in clip_sentences:
            rows_list.append({'Sentense': sentence, 'Tag': int(is_interesting)})

        dataset = pd.concat([dataset, pd.DataFrame(rows_list)]).reset_index(drop=True)
        dataset.to_csv('dataset.csv', index=False)
    
dataset.to_csv('dataset.csv', index=False)